In [2]:
import pandas as pd

## My API key  & library_Id are stored in another file
api_key,library_id = pd.read_json("Zotero_API_key.json", typ='series')

library_type="group"




from pyzotero import zotero
zot = zotero.Zotero(library_id, library_type, api_key)

ID_Universities='EWSYI3RS' # to access the files in the university subfolder.

In [3]:
for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")

name: Radboud Nijmegen --> key: 5JQEUPHJ
name: Other missing --> key: TRP43B6E
name: Leiden University --> key: P483KEM8
name: OpenUniversiteitNederland --> key: X4HX8ZQE
name: Wageningen University --> key: QMTMEV7G
name: University of Twente --> key: QFGTICY2
name: Utrecht University --> key: 9A2UJIVA
name: Rotterdam University --> key: Z4EJ3VVG
name: University of Groningen --> key: P63XN58M
name: Vrije Universiteit Amsterdam --> key: AZJ9ZEBZ
name: Tilburg University --> key: XF9572P7
name: Eindhoven University of Technoloy --> key: FKNPJ5UD
name: Maastricht Univerisity --> key: QW34VSNN
name: Delft University --> key: FZDYP465
name: University of Amsterdam --> key: 92FECCMX


In [4]:
def relevantItemInfo(dictionary):    
    if dictionary["itemType"]=='attachment':
        # if directory --> key leads to the correct folder
        
        relevant_keys=['key','url']
        newDict={key: dictionary[key] for key in relevant_keys}
        #print(dictionary)
        newDict["note"]=None
       # print(newDict)
        return newDict
    
    if dictionary["itemType"]=="note":
        relevant_keys=['key','note']
        newDict={key: dictionary[key] for key in relevant_keys}
        newDict["url"]=None
        return newDict

    

for i in zot.collection_items(key)[:2]:
    dictionary=i["data"]
    print(dictionary, end="\n\n")
    
    Info=relevantItemInfo(dictionary)
    print(Info, end="\n\n")

{'key': '7GX265HI', 'version': 1373, 'parentItem': 'ISJFJ7MB', 'itemType': 'attachment', 'linkMode': 'imported_url', 'title': 'News and Events: Open Positions | Institute for Logic, Language and Computation', 'accessDate': '2023-06-05T17:06:57Z', 'url': 'https://www.illc.uva.nl/NewsandEvents/News/Positions/newsitem/13993/Assistant-Professor-in-Generative-AI-in-the-Arts', 'note': '', 'contentType': 'text/html', 'charset': 'utf-8', 'filename': 'Assistant-Professor-in-Generative-AI-in-the-Arts.html', 'md5': None, 'mtime': None, 'tags': [], 'relations': {'dc:replaces': 'http://zotero.org/groups/5048691/items/RA6K6BCG'}, 'dateAdded': '2023-06-05T17:06:57Z', 'dateModified': '2023-06-05T17:34:31Z'}

{'key': '7GX265HI', 'url': 'https://www.illc.uva.nl/NewsandEvents/News/Positions/newsitem/13993/Assistant-Professor-in-Generative-AI-in-the-Arts', 'note': None}

{'key': 'ISJFJ7MB', 'version': 1373, 'itemType': 'webpage', 'title': 'News and Events: Open Positions | Institute for Logic, Language an

In [5]:
dictList=[]

for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")
    
    for i in zot.collection_items(key):
        # only use data if there is an attachment.
        if i["data"]["itemType"]=='attachment':
            Info=relevantItemInfo(i["data"])
            dictList+=[{"Uni": name,"FileKey":Info["key"],"url":Info["url"]}]
    


name: Radboud Nijmegen --> key: 5JQEUPHJ
name: Other missing --> key: TRP43B6E
name: Leiden University --> key: P483KEM8
name: OpenUniversiteitNederland --> key: X4HX8ZQE
name: Wageningen University --> key: QMTMEV7G
name: University of Twente --> key: QFGTICY2
name: Utrecht University --> key: 9A2UJIVA
name: Rotterdam University --> key: Z4EJ3VVG
name: University of Groningen --> key: P63XN58M
name: Vrije Universiteit Amsterdam --> key: AZJ9ZEBZ
name: Tilburg University --> key: XF9572P7
name: Eindhoven University of Technoloy --> key: FKNPJ5UD
name: Maastricht Univerisity --> key: QW34VSNN
name: Delft University --> key: FZDYP465
name: University of Amsterdam --> key: 92FECCMX


In [6]:
df=pd.DataFrame(dictList)
df.head()

,Uni,FileKey,url
0,Radboud Nijmegen,QJ4WZUBB,https://www.ru.nl/communicationscience/researc...
1,Radboud Nijmegen,K3P5X7U4,https://www.ru.nl/communicatiewetenschap/onder...
2,Radboud Nijmegen,7IIQTU72,
3,Radboud Nijmegen,TUS48YKW,
4,Radboud Nijmegen,U97FGRHV,https://www.ru.nl/@1235768/4-revolutionaire-te...


In [7]:
def FileLinks(Filekey):
   #print(Filekey)
    import os
    DIR=f"C:\\Users\\mr\\Zotero\\storage\\{Filekey}"
    #print(DIR)
    FileNames=os.listdir(DIR)
    file=[i for i in FileNames if i.endswith(".html")]
    return DIR+"\\"+FileNames[-1]



df["file"]=df.FileKey.apply(FileLinks)
df.head()

,Uni,FileKey,url,file
0,Radboud Nijmegen,QJ4WZUBB,https://www.ru.nl/communicationscience/researc...,C:\Users\mr\Zotero\storage\QJ4WZUBB\research-p...
1,Radboud Nijmegen,K3P5X7U4,https://www.ru.nl/communicatiewetenschap/onder...,C:\Users\mr\Zotero\storage\K3P5X7U4\intoeat-ch...
2,Radboud Nijmegen,7IIQTU72,,C:\Users\mr\Zotero\storage\7IIQTU72\Bouschery ...
3,Radboud Nijmegen,TUS48YKW,,C:\Users\mr\Zotero\storage\TUS48YKW\Merkx et a...
4,Radboud Nijmegen,U97FGRHV,https://www.ru.nl/@1235768/4-revolutionaire-te...,C:\Users\mr\Zotero\storage\U97FGRHV\4-revoluti...


In [8]:
df.to_json("2023-06-05_Uni_Files_raw.json")

In [ ]:
df=pd.read_json("2023-05-10_Uni_Files_raw.json")

# Add Content from HTMLs

In [9]:
from htmldate import find_date
#find_date("https://www.uu.nl/en/events/chat-gpt-discover-the-magic-of-ai")

def find_dateX(x):
    try:
        return find_date(x)
    except:
        print(x)


df["date"]=df.url.apply(find_dateX)

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9900745/pdf/nihpp-2023.01.21.525030v1.pdf
https://www.ou.nl/en/-/symposium-towards-highly-informative-learning-analytics?p_l_back_url=%2Fen%2Fzoekresultaten%3Fq%3DChatGPT
https://www.ou.nl/en/-/symposium-towards-highly-informative-learning-analytics?p_l_back_url=%2Fen%2Fzoekresultaten%3Fq%3DChatGPT
https://www.sciencedirect.com/science/article/pii/S000326702101134X/pdfft?md5=5eb1e80b2408d3309fa72ee46a08ff68&pid=1-s2.0-S000326702101134X-main.pdf&isDTMRedir=Y
http://library.wur.nl/WebQuery/wurpubs/fulltext/456019


C:\Users\mr\miniconda3\envs\PyMax\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


https://edepot.wur.nl/456019
https://www.sciencedirect.com/science/article/pii/S2515856220300122/pdf?md5=f1761b375e601ac6c194c6740c4c736e&pid=1-s2.0-S2515856220300122-main.pdf&isDTMRedir=Y


C:\Users\mr\miniconda3\envs\PyMax\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


https://sci-hub.se/downloads/2019-10-18/3f/10.1007@978-3-030-33778-0.pdf#navpanes=0&view=FitH
https://publications.aaahq.org/iae/article-pdf/doi/10.2308/ISSUES-2023-013/90291/issues-2023-013.pdf
https://publications.aaahq.org/iae/article-pdf/doi/10.2308/ISSUES-2023-013/90291/issues-2023-013.pdf
https://theconversation.com/debate-chatgpt-offers-unseen-opportunities-to-sharpen-students-critical-skills-199264
https://vumagazine.vu.nl/the-west-will-have-to-run-just-to-keep-up-with-countries-like-china
https://publications.aaahq.org/iae/article-pdf/doi/10.2308/ISSUES-2023-013/90291/issues-2023-013.pdf
https://publications.aaahq.org/iae/article-pdf/doi/10.2308/ISSUES-2023-013/90291/issues-2023-013.pdf
https://universonline.nl/nieuws/2016/04/28/stage-ook-voor-academici-steeds-belangrijker/
https://universonline.nl/nieuws/2023/03/27/education-after-covid-19-back-to-the-lecture-halls-or-is-online-education-here-to-stay/
https://universonline.nl/nieuws/2023/03/21/onderwijs-na-corona-terug-naar-d

In [10]:
import MyLib.nlp as nlp
import MyLib.HTML_prep as HTML_prep
import MyLib.analysis as analysis 
%load_ext autoreload
%autoreload 2

In [260]:

def FileInfo(FilePath):
    Title,h1,h2,text,linkName,linkUrl=None,None,None,None,None,None
    if FilePath.endswith(".html"):
    
        try:
            #print(FilePath)

            with open(FilePath,"r", encoding='utf-8') as f:
                text= f.read()
            
        except:
            print("error opening the html file. File does not exist?")
            return
        
        
        try:
            from bs4 import BeautifulSoup

            soup = BeautifulSoup(text, 'html.parser')
            
            
            
            Title=soup.title.text
            #soup=HTML_prep.removeIMGs(soup,IMG_filename=Title.replace(" ","_"))      


            All_divs=[p.parent for p in soup("p") if p.parent.name=="div"]
            All_divs=(list(set(All_divs)))
            
            
            
            # Remove images
            try:
            
                IMG_filename="".join(x for x in Title.strip().replace(" ","_") if x.isalnum() or x=="_")[:100]
                All_divs=[HTML_prep.removeIMGs(div,write_img=False,IMG_filename=IMG_filename) for div in All_divs]        

            except:
                print(f"error removing images: {FilePath}")

            

            h1=[item for sublist in [[i.text for i in div("h1")] for div in All_divs if div("h1")!=None] for item in sublist]
            h2=[item for sublist in [[i.text for i in div("h2")] for div in All_divs if div("h2")!=None] for item in sublist]
            text=[item for sublist in [[i.text for i in div("p")] for div in All_divs if div("p")!=None] for item in sublist]
            
    
            linkName=[item for sublist in [[i.text for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            linkUrl=[item for sublist in [[i.get('href') for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]


        except:
            print(f"error with: {FilePath}", end=". ")
            
            try:
                text=text.split("\n\n")
 
                print("--> text from reading as a text file.")
            except:
                print("also no text file")
                
        #leave out words longer than 100 characters.
        text=[" ".join([y.replace("\n"," ") for y in i.split(" ") if len(y)<100]) for i in text] 

            
    return pd.Series([Title,h1,h2,text,linkName,linkUrl])
        
        

#df.file[2:6].apply(FileInfo)
df[["Title","h1","h2","text","linkName","linkUrl"]]=df.file.apply(FileInfo)


error with: C:\Users\mr\Zotero\storage\RD8VNVZD\nrc-digitaledieven-23-02-08.html. --> text from reading as a text file.
error removing images: C:\Users\mr\Zotero\storage\MDTRC7HH\AI In Education.html
error with: C:\Users\mr\Zotero\storage\JHV5G985\url.html. --> text from reading as a text file.


In [213]:
FilePath="C:\Users\mr\Zotero\storage\MDTRC7HH\AI In Education.html"

with open(FilePath,"r", encoding='utf-8') as f:
    text= f.read()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3671691020.py, line 1)

In [305]:
df[df.FileKey=="G9FMK9GK"]["text"][324][6][:5000]

'In the field of cognitive psychology, the human mind is often referred to as a "cognitive miser" because people, regardless of their intelligence, tend to choose simpler and easier thinking methods instead of more complex and cognitively demanding ones (Stanovich, 2009). The underlying assumption of the concept is that humans are limited in their capacity to process information, so they take shortcuts whenever they can (Fiske & Taylor, 1991; Kahneman, 2003). The cognitive miser theory, first introduced by Fiske & Taylor (1991), suggests that people engage in economically cost-effective thought processes instead of rationally weighting costs against benefits and updating expectations based upon the results of their everyday actions (Fiske & Taylor, 1991).Much of the cognitive miser theory is built upon research done on heuristics in human judgment and decision-making. For instance, dual-process theories explain how human thought processes are mainly distinguished: System 1, being intui

In [269]:
df.to_json("2023-06-05_Uni_Files_NLP.json")

In [369]:
df=pd.read_json("2023-06-05_Uni_Files_NLP.json")



In [370]:
def Insert_title(text,Title):
    if Title!=None and type(text)==list:
        text=[Title]+text
        return text
    else:
        return text


df["text"]=df[["text","Title"]].apply(lambda x: Insert_title(*x),axis=1)

In [371]:

def Splitter(paragraphs):
    split_paragraphs=[]
    for paragraph in paragraphs:
        l=len(paragraph)
        import math
        times=int(math.ceil(l/5000))
        
        if times==1:
            split_paragraphs.append(paragraph)
        
        if times>1:
            split_paragraphs.append(paragraph[0:4999])
            
            for n in range(1,times-1):
                split_paragraphs.append(paragraph[(n*5000):((n+1)*5000-1)])
                
    return split_paragraphs

ChatGPT_Terms="ChatGPT, Chat-GPT, GPT3, GPT-3, GPT-x, GPT-4, GPT4, Transformer, OpenAI, AI, hallucination, Text generation, LLM, A.I, GPT, Chatbot, technology, tool, app, Artificial Intelligence, Large Language Models, generative AI"

df["AI_paragraphs"]=df.text.apply(nlp.filter_paragraphs,by=ChatGPT_Terms).dropna()

df["AI_paragraphs_len"]=df["AI_paragraphs"].dropna().apply(lambda x: [len(i) for i in x])

df["AI_paragraphs"]=df.AI_paragraphs.dropna().apply(Splitter)

#df[df.AI_Paragraphs_len.apply(lambda x: any(i > 4999 for i in x))]


In [372]:
column="AI_paragraphs"

df=df[df.file.apply(lambda x: x.endswith("html"))]

df=df[df.Title!='Research Posters - Faculty of Geosciences - Utrecht University']

df=df.explode(column).reset_index(drop=True)
df=df[df[column].apply(lambda x: type(x)==str)]
df=df.drop_duplicates(column)
df["LEN"]=df[column].apply(lambda x: len(x.split(" ")))


In [375]:
df

,Uni,FileKey,url,file,date,Title,h1,h2,text,linkName,linkUrl,AI_paragraphs,AI_paragraphs_len,LEN,AI_paragraphs_len2
0,Radboud Nijmegen,QJ4WZUBB,https://www.ru.nl/communicationscience/researc...,C:\Users\mr\Zotero\storage\QJ4WZUBB\research-p...,2023-01-01,Research projects - Communication Science,[],[],"[Research projects - Communication Science, Co...","[More information., Information Explosion, INt...",[https://www.ru.nl/communicationscience/@96084...,INtoEAT: Chatbot algorithms and coaching strat...,"[69, 106]",9,69
1,Radboud Nijmegen,QJ4WZUBB,https://www.ru.nl/communicationscience/researc...,C:\Users\mr\Zotero\storage\QJ4WZUBB\research-p...,2023-01-01,Research projects - Communication Science,[],[],"[Research projects - Communication Science, Co...","[More information., Information Explosion, INt...",[https://www.ru.nl/communicationscience/@96084...,Chatbot Usability - Studying the Impact of Cha...,"[69, 106]",15,106
2,Radboud Nijmegen,K3P5X7U4,https://www.ru.nl/communicatiewetenschap/onder...,C:\Users\mr\Zotero\storage\K3P5X7U4\intoeat-ch...,2023-01-01,INtoEAT: Chatbot algorithms and coaching strat...,[],[],[INtoEAT: Chatbot algorithms and coaching stra...,[],[],INtoEAT: Chatbot algorithms and coaching strat...,"[94, 867, 479]",11,94
3,Radboud Nijmegen,K3P5X7U4,https://www.ru.nl/communicatiewetenschap/onder...,C:\Users\mr\Zotero\storage\K3P5X7U4\intoeat-ch...,2023-01-01,INtoEAT: Chatbot algorithms and coaching strat...,[],[],[INtoEAT: Chatbot algorithms and coaching stra...,[],[],A healthy diet can help keep the body and mind...,"[94, 867, 479]",132,867
4,Radboud Nijmegen,K3P5X7U4,https://www.ru.nl/communicatiewetenschap/onder...,C:\Users\mr\Zotero\storage\K3P5X7U4\intoeat-ch...,2023-01-01,INtoEAT: Chatbot algorithms and coaching strat...,[],[],[INtoEAT: Chatbot algorithms and coaching stra...,[],[],Research within the FSW and BSI combines insig...,"[94, 867, 479]",69,479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,University of Amsterdam,ZF4F4VKR,https://tlc.uva.nl/article/de-impact-van-chatg...,C:\Users\mr\Zotero\storage\ZF4F4VKR\de-impact-...,NaT,De impact van ChatGPT op het onderwijs - UvA T...,[],"[, Hoe check je of je opdrachten en beoordelin...",[De impact van ChatGPT op het onderwijs - UvA ...,"[OpenAI, Contact TLC, FAQ, deze tips om prompt...","[https://openai.com/, https://tlc.uva.nl/conta...",Studenten kunnen de chatbot wel voor andere do...,"[74, 551, 558, 571, 384, 350, 170, 237, 162]",56,384
1885,University of Amsterdam,ZF4F4VKR,https://tlc.uva.nl/article/de-impact-van-chatg...,C:\Users\mr\Zotero\storage\ZF4F4VKR\de-impact-...,NaT,De impact van ChatGPT op het onderwijs - UvA T...,[],"[, Hoe check je of je opdrachten en beoordelin...",[De impact van ChatGPT op het onderwijs - UvA ...,"[OpenAI, Contact TLC, FAQ, deze tips om prompt...","[https://openai.com/, https://tlc.uva.nl/conta...",Op dit moment is het gebruik van ChatGPT soms ...,"[74, 551, 558, 571, 384, 350, 170, 237, 162]",59,350
1886,University of Amsterdam,ZF4F4VKR,https://tlc.uva.nl/article/de-impact-van-chatg...,C:\Users\mr\Zotero\storage\ZF4F4VKR\de-impact-...,NaT,De impact van ChatGPT op het onderwijs - UvA T...,[],"[, Hoe check je of je opdrachten en beoordelin...",[De impact van ChatGPT op het onderwijs - UvA ...,"[OpenAI, Contact TLC, FAQ, deze tips om prompt...","[https://openai.com/, https://tlc.uva.nl/conta...",Veel van de informatie die ChatGPT genereerd i...,"[74, 551, 558, 571, 384, 350, 170, 237, 162]",27,170
1887,University of Amsterdam,ZF4F4VKR,https://tlc.uva.nl/article/de-impact-van-chatg...,C:\Users\mr\Zotero\storage\ZF4F4VKR\de-impact-...,NaT,De impact van ChatGPT op het onderwijs - UvA T...,[],"[, Hoe check je of je opdrachten en beoordelin...",[De impact van ChatGPT op het onderwijs - UvA ...,"[OpenAI, Contact TLC, FAQ, deze tips om prompt...","[https://openai.com/, https://tlc.uva.nl/conta...",Je zou dit kunnen onderzoeken door de chatbot ...,"[74, 551, 558, 571, 384, 350, 170, 237, 162]",40,237


In [373]:
df["AI_paragraphs_len2"]=df["AI_paragraphs"].dropna().apply(lambda x: len(x))

df[df["AI_paragraphs_len2"]>4000]

,Uni,FileKey,url,file,date,Title,h1,h2,text,linkName,linkUrl,AI_paragraphs,AI_paragraphs_len,LEN,AI_paragraphs_len2
210,Leiden University,8BXX68RU,https://www.leidenlearninginnovation.org/stori...,C:\Users\mr\Zotero\storage\8BXX68RU\chatgpt-ex...,2023-03-08,\n Explore ChatGPT's Potential for Higher Ed: ...,[Explore ChatGPT’s Potential for Higher Ed: Tr...,[Insights],[\n Explore ChatGPT's Potential for Higher Ed:...,"[Collegecolumn: ChatGPT, een bedreiging of een...",[https://www.medewerkers.universiteitleiden.nl...,In this exercise you’ll play around with ChatG...,"[625, 398, 784, 569, 224, 4936, 4608, 4303, 40...",719,4936
211,Leiden University,8BXX68RU,https://www.leidenlearninginnovation.org/stori...,C:\Users\mr\Zotero\storage\8BXX68RU\chatgpt-ex...,2023-03-08,\n Explore ChatGPT's Potential for Higher Ed: ...,[Explore ChatGPT’s Potential for Higher Ed: Tr...,[Insights],[\n Explore ChatGPT's Potential for Higher Ed:...,"[Collegecolumn: ChatGPT, een bedreiging of een...",[https://www.medewerkers.universiteitleiden.nl...,"Before you dive into the exercises, navigate t...","[625, 398, 784, 569, 224, 4936, 4608, 4303, 40...",663,4608
212,Leiden University,8BXX68RU,https://www.leidenlearninginnovation.org/stori...,C:\Users\mr\Zotero\storage\8BXX68RU\chatgpt-ex...,2023-03-08,\n Explore ChatGPT's Potential for Higher Ed: ...,[Explore ChatGPT’s Potential for Higher Ed: Tr...,[Insights],[\n Explore ChatGPT's Potential for Higher Ed:...,"[Collegecolumn: ChatGPT, een bedreiging of een...",[https://www.medewerkers.universiteitleiden.nl...,We’ve created sample prompts for you. Use them...,"[625, 398, 784, 569, 224, 4936, 4608, 4303, 40...",611,4303
213,Leiden University,8BXX68RU,https://www.leidenlearninginnovation.org/stori...,C:\Users\mr\Zotero\storage\8BXX68RU\chatgpt-ex...,2023-03-08,\n Explore ChatGPT's Potential for Higher Ed: ...,[Explore ChatGPT’s Potential for Higher Ed: Tr...,[Insights],[\n Explore ChatGPT's Potential for Higher Ed:...,"[Collegecolumn: ChatGPT, een bedreiging of een...",[https://www.medewerkers.universiteitleiden.nl...,Play around with the prompts. Feel free to rep...,"[625, 398, 784, 569, 224, 4936, 4608, 4303, 40...",571,4061
250,Leiden University,QZMK2D73,https://www.intermediair.nl/werk-en-carriere/l...,C:\Users\mr\Zotero\storage\QZMK2D73\deze-gewil...,2023-06-05,Deze gewilde banen dreigen (niet) te verdwijne...,[],"[Gevarenzone, Routinewerk eerder vervangen, Ch...",[Deze gewilde banen dreigen (niet) te verdwijn...,"[onderzocht Nationale Vacaturebank, Horizon Eu...",[https://www.nationalevacaturebank.nl/carriere...,De vraag welke banen door automatisering overb...,"[5432, 4760, 3504, 2939, 2423, 1703, 1270, 792]",690,4999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,Eindhoven University of Technoloy,9QZIXMJN,https://www.cursor.tue.nl/nieuws/2023/mei/week...,C:\Users\mr\Zotero\storage\9QZIXMJN\brainport-...,2023-05-04,"""Brainport heeft meer software-focus nodig""","[""Brainport heeft meer software-focus nodig"", ...",[],"[""Brainport heeft meer software-focus nodig"", ...","[geven, in te schrijven, \n\n, \n\n, \n\n, \n\...","[https://cui.acm.org/2023/programme/keynotes/,...",Wat als Word tegen je zou gaan praten? “Clippy...,"[4558, 3873, 3167, 2622, 1866, 1613, 1603, 822]",725,4558
1419,Eindhoven University of Technoloy,QRFXZFNJ,https://www.cursor.tue.nl/achtergrond/2023/apr...,C:\Users\mr\Zotero\storage\QRFXZFNJ\ai-heeft-d...,2023-04-26,"“AI heeft de potentie de mensheid te redden, m...",[\n Cursor-nieuwsbrief\n ],"[""Het bewustzijn in de sector is gegroeid en d...","[“AI heeft de potentie de mensheid te redden, ...","[\n\n, \n\n, \n\n, \n\n, \n\n, \n\n, myTomorro...",[https://www.facebook.com/sharer/sharer.php?u=...,"Daniel Kapitan, fellow bij EAISI, ondertekende...","[573, 1356, 790, 5999, 5261, 4472, 3873, 3471,...",814,4999
1420,Eindhoven University of Technoloy,QRFXZFNJ,https://www.cursor.tue.nl/achtergrond/2023/apr...,C:\Users\mr\Zotero\storage\QRFXZFNJ\ai-heeft-d...,2023-04-2

In [376]:
df=nlp.NLP_Pipeline(df, sentiment=False, language="en",translate=True, column=column)

Current Time = 15:20:02
Lenght:  1622
cleaning done.
Current Time = 15:20:03
Current Time = 15:20:11
language detection done.
Translating...
nl Over de mo
nl ‘Een aantr
nl De ene tec
nl ChatGPT ge
nl Volgens so
nl ChatGPT ge
nl Zelfs de l
nl   ChatGPT 
nl EdTech dag
nl Hoe slim h
nl Er is al v
nl AI kan de 
nl Er komen o
nl Wil je (bl
nl Meestal he
nl Nooit meer
nl Er zijn tw
nl Het onderz
nl Maar, cons
nl Wat beteke
nl Artificiël
nl "Wat vindt
nl De antwoor
nl ChatGPT is
nl Na Plato’s
nl ChatGPT pl
nl Een chatbo
nl Gaat GPT-4
nl De ontwikk
nl Het intern
nl In deze zo
nl Kom en den
nl Extra lang
nl Large Lang
nl Salonlezin
nl Bij de rec
nl Kan AI een
nl Overheerst
nl Een chatbo
nl De chatbot
nl AI in het 
nl Wat kunnen
nl ‘We bevind
nl Workshop C
nl Op donderd
nl De komst v
nl Leidse stu
nl Het viel h
nl In hoeverr
nl In een rec
nl ChatGPT en
nl ChatGPT ge
nl Hoewel AI-
nl Geheel voo
nl chatgpt in
nl De nieuwe 
nl Gebruik ji
nl Het progra
nl Hoewel pro
nl We moeten 
nl Wij onders
nl De

In [380]:
df.columns

df2=df[['Uni','FileKey','url','date', 'Title','linkName', 'linkUrl', 'AI_paragraphs','text_clean', 'letters_count', 'word_count',
       'language', 'source_language', 'pure_text', 'Lemmata', 'NoStopwords']]

In [14]:
#df2.text=df2["AI_paragraphs"].apply(lambda x:" ".join(x))

In [ ]:
df2[df2.text.apply(lambda x: type(x))!=str]

In [ ]:
df2["LEN"]=df2.text.apply(lambda x: len(x))
df2=df2[df2.LEN<df2.LEN.max()] ###Cut out the very large list

In [ ]:
df2.LEN.describe()

In [ ]:
df2=df.reset_index(drop=False)

In [381]:
df2.to_json("2023-06-06_Zotero_AI_nlp_en2.json")

In [67]:
from ipywidgets import interactive, interact

@interact(column=["NoStopwords"],Mostcommon=(5,200))
def make_wordcloud(column="NoStopwords",Mostcommon=20,Save_as=""):
    hashtags=[i.lower() for s in df[column].dropna() for i in s] # hashtags OR # no stopwords
    analysis.make_wordcloud(hashtags,filename=Save_as,Mostcommon=Mostcommon)

interactive(children=(Dropdown(description='column', options=('NoStopwords',), value='NoStopwords'), IntSlider…

In [ ]:
df3.to_excel("2023-05-08_ZoteroHTMLs.xlsx")